In [28]:
from bokeh.models import (ColumnDataSource, GMapPlot, Range1d, ColumnDataSource, LinearAxis,
HoverTool, PanTool, WheelZoomTool, BoxZoomTool, ResetTool, CustomJS, Slider,
GMapOptions, NumeralTickFormatter, PrintfTickFormatter, MapOptions)
from bokeh.io import output_file
from bokeh.plotting import figure, save, show, reset_output
from bokeh.tile_providers import CARTODBPOSITRON
from bokeh.io import output_file, show
from bokeh.layouts import column
import pandas as pd
import pyproj
import requests
from io import StringIO
import numpy as np

outfp = r"./index.html"
output_file(outfp)

wgs84=pyproj.Proj("+init=EPSG:4326") # LatLon with WGS84 datum used by GPS units and Google Earth
webdb=pyproj.Proj("+init=EPSG:3857") # Web Mercator

zoom = 15000 
washington_y = 38.904978
washington_x = -77.039658

x, y = pyproj.transform(wgs84, webdb, washington_x, washington_y)
x_scroll = Range1d(x-zoom,x+zoom, bounds="auto") # bounds
y_scroll = Range1d(y-zoom,y+zoom, bounds="auto") # bounds

tool_tips = [("email", "@email"),
("home address", "@home"),
("have nanny", "@have_nanny"),
("child age", "@youngest_age"),
("number children", "@number_children"),
("time range", "@time_range" )]

wheel = WheelZoomTool()
tool_box = [HoverTool(tooltips=tool_tips), 
            wheel, PanTool(), ResetTool()]
reset_output()

plot = figure(x_range=x_scroll,
    y_range=y_scroll,
    toolbar_location="right",
    plot_width=800,
    plot_height=800,
    tools=tool_box)

plot.toolbar.active_scroll = wheel
plot.add_tile(CARTODBPOSITRON)

df = pd.read_csv('./data.csv')
if 'Latitude ' in df.columns:
    df.rename({'Latitude ': 'Latitude'}, inplace=True, axis=1)
df.columns = ['email', 'home', 'city', 'state', 'zip', 
              'latitude', 'longitude', 'have_nanny', 
              'youngest_age', 'number_children', 
              'time_range']
df = df.dropna(how='any')
conversions = np.array([pyproj.transform(wgs84, webdb, x, y) for x, y in zip(df.longitude, df.latitude)])
df['y_points'] = conversions[:,1]
df['x_points'] = conversions[:,0]

dsource=ColumnDataSource(df[['x_points', 'y_points', 'email', 'home', 'have_nanny',
                             'youngest_age', 'number_children', 'time_range']])
points = plot.circle('x_points', 'y_points', fill_color="white", size=6, source=dsource,
                     tags = ['email', 'home', 'have_nanny', 'youngest_age', 'number_children', 'time_range'])
show(plot)

# def limit_selection(source=dsource, window=None):
#     df=source.data
# #     f = cb_obj.value
# #     if f==0:
#     df=df[df['time_range']=='next 30 days']
#     dsource.change.emit()
   
# slider = Slider(start=0, end=1, value=1, step=1, title="toggle",
#                 callback=CustomJS.from_py_func(limit_selection))
# layout = column(slider, plot)
# show(layout)